## dataset 분석
input : 모터에 부착된 전류 센서에서 4kHz로 측정된  전류 정보 (R, S, T상 전류) 값, Train Set에는 정상/고장에 대한 레이블(label) 정보 없음
- wavelength : 시간
- x : R성 전류
- y : S성 전류
- z : T성 전류

output : 정상 / 고장 

label 정보가 없기 때문에 비지도학습으로 output을 도출해야한다.

In [7]:
import pandas as pd
import numpy as np
import os
import math
import pickle
from mpl_toolkits.mplot3d import axes3d

import matplotlib.pyplot as plt

In [3]:
def middle_line(temp_df) :
    x = temp_df.x
    y = temp_df.y
    z = temp_df.z
    return math.sqrt((x + y*(-0.5) + z*(-0.5))**2 + (y*(0.5*math.sqrt(3)) + z*(-0.5*math.sqrt(3)))**2)
def caculate_unbalance(temp_df) :
    x = temp_df.x
    y = temp_df.y
    z = temp_df.z
    return int((max(x, y, z) - min(x, y, z))/sum([x, y, z])*3 * 100)
train_path = 'E:/anomoly_detection_IOT_sensor/iotAnomaly_train'

dic = {'중성선' : [], 'path' : [], '전류 불평형율' : [] }
for (path, dir, files) in os.walk(train_path):
    print(path)
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.csv':
            df = pd.read_csv("%s/%s" % (path, filename))
            df['중성선'] = df[['x', 'y', 'z']].apply(middle_line, axis=1)
            df['불평형률'] = df[['x', 'y', 'z']].apply(caculate_unbalance, axis=1)
            dic['path'].append("%s/%s" % (path, filename))
            
            
            break
df

E:/anomoly_detection_IOT_sensor/iotAnomaly_train
E:/anomoly_detection_IOT_sensor/iotAnomaly_train\csv_files


,wavelength,x,y,z,중성선,불평형률
0,0.0000,2.692627,0.450439,-3.553467,5.480527,-4565
1,0.0005,2.132080,1.291260,-3.833740,5.593014,-4360
2,0.0010,1.731689,1.931885,-4.114014,5.948328,-4026
3,0.0015,0.930908,2.332275,-3.673584,5.442220,-4390
4,0.0020,0.330322,2.932861,-3.633545,5.727266,-5318
...,...,...,...,...,...,...
1995,0.9975,4.334229,-2.712646,-2.112061,6.766602,-4310
1996,0.9980,4.534424,-1.871826,-3.193115,7.158935,-4369
1997,0.9985,4.053955,-0.990967,-3.593506,6.734542,-4324
1998,0.9990,3.613525,0.010010,-4.073975,6.661903,-5120


중성선이 0에 가까울수록 정상일 확률이 높고, 불평형률이 30% 이상이라면 고장날 확률이 높다.

이 기준으로 고장났거나 고장날 확률이 높은지 분류하기 위해 각 전류에 위상차를 표현하기 위해 위와 같은 계산을 하였다.

그런데 대부분의 데이터에서 0과 가깝게 나오지 않았다.

하지만 단순히 **x+y+z**를 하면 0과 근사한 값이 나온다.

(사실, A로 나타나려면 음수가 나올 수가 없다...)

따라서 이 데이터가 이미 위상차 계산이 되어있다고 가정하고 프로젝트를 진행하였다.

참고 : 
- [1] [중성선의 원리 및 특징, 평형과 불평형 전류 이해하기](https://blog.naver.com/somang8991/221916506560)
- [2] [3상 4선식에서 중성선에 흐르는 전류를 그림으로 그렸습니다.](https://m.blog.naver.com/kimih2917/40122115829)
- [3] [3상 R,S,T 불평형](https://m.blog.naver.com/sangdoo7572/221508664048)

------------------

### 지도학습
지도학습이라면
1. Support vector machine
2. Bayesian network
3. K-nearest neighbors
가 이상 탐지 기법에 적용 가능하다.

지도학습의 전제 조건 : 균형 데이터일 때, 경계가 분명할 때 사용해야한다. 
+ 불균형 데이터일때 : 특정 label만 도출할 수 있다.
+ 경계가 불분명할 때 : 분류했을 때 경계가 불분명한 데이터는 잘못된 label을 낼 가능성이 있다.

### 비지도학습 

지도학습 알고리즘을 적용하기 어려울 때 사용하며, **unlabeled data**에서 데이터의 구조(structure)와 패턴(pattern)을 알아내는 문제의 범주에 속한다.

정상, 이상에 대한 라벨링이 힘들 것으로 예상될때도 적용할 수 있고, 특히 데이터에 **이상치가 매우 적게 나타날때** 유용한 기법이다.

- Clustering
- Principal Component Analysis (PCA)
- Autoencoder

### 시계열

해당 데이터는 시간에 따라 데이터의 변화를 나타낸다.

따라서 RNN, LSTM 등의 시퀀스(순서) 데이터 처리가 가능한 알고리즘을 결합하여 사용하면 높은 정확도를 구할 수 있을것이다.


평가지표

AUROC(Area Under an ROC Curve)

     - 평가를 위한 Test Set에는 정상/고장 여부 label 존재
     - ROC (Receiver Operating Characteristic) 곡선 아래 영역의 넓이로 모델의 정확도 측정

## dataset 저장

### train data

In [5]:
train_path = 'E:/anomoly_detection_IOT_sensor/iotAnomaly_train'
dic_minmax = {'x_max' : [], 'y_max' : [], 'z_max' : [],
              'x_min' : [], 'y_min' : [], 'z_min' : [],
              'sum_max' : [], 'sum_min' : [], 'path' : [] }
for (path, dir, files) in os.walk(train_path):
    print(path)
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.csv':
            df = pd.read_csv("%s/%s" % (path, filename))
            df['sum'] = df['x'] + df['y'] + df['z']
            dic_minmax['x_max'].append(df.x.max())
            dic_minmax['y_max'].append(df.y.max())
            dic_minmax['z_max'].append(df.z.max())
            dic_minmax['x_min'].append(df.x.min())
            dic_minmax['y_min'].append(df.y.min())
            dic_minmax['z_min'].append(df.z.min())
            dic_minmax['sum_max'].append(df['sum'].max())
            dic_minmax['sum_min'].append(df['sum'].min())
            dic_minmax['path'].append("%s/%s" % (path, filename))
            
# 저장된 데이터 개수 확인
print( dic_minmax.keys() )
print( len(dic_minmax['path']) )

E:/anomoly_detection_IOT_sensor/iotAnomaly_train
E:/anomoly_detection_IOT_sensor/iotAnomaly_train\csv_files


58631

### test data

In [ ]:
test_path = 'E:/anomoly_detection_IOT_sensor/iotAnomaly_test'
dic_minmax = {'x_max' : [], 'y_max' : [], 'z_max' : [],
              'x_min' : [], 'y_min' : [], 'z_min' : [],
              'sum_max' : [], 'sum_min' : [], 'path' : [] }
for (path, dir, files) in os.walk(test_path):
    print(path)
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.csv':
            try :
                df = pd.read_csv("%s/%s" % (path, filename))
                df['sum'] = df['x'] + df['y'] + df['z']
                dic_minmax['x_max'].append(df.x.max())
                dic_minmax['y_max'].append(df.y.max())
                dic_minmax['z_max'].append(df.z.max())
                dic_minmax['x_min'].append(df.x.min())
                dic_minmax['y_min'].append(df.y.min())
                dic_minmax['z_min'].append(df.z.min())
                dic_minmax['sum_max'].append(df['sum'].max())
                dic_minmax['sum_min'].append(df['sum'].min())
                dic_minmax['path'].append("%s/%s" % (path, filename))
            except KeyError as e :
                print("%s/%s" % (path, filename))
                break
                
# 저장된 데이터 개수 확인
print( dic_minmax.keys() )
print( len(dic_minmax['path']) )

## pickling ( 저장 )

각 파일(csv)의 열마다 max, min값을 저장한 dictionary를 다른 파일에서도 사용할 수 있도록 local에 저장하자

In [8]:
# train_data 저장
with open("dic_minmax.pickle", "wb") as handle:
    pickle.dump(dic_minmax, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
# 저장 잘 됐는지 확인
with open("dic_minmax.pickle", "rb") as handle:
    f = pickle.load(handle)

print( f.keys() )
print( len(f['path']) )

dict_keys(['x_max', 'y_max', 'z_max', 'x_min', 'y_min', 'z_min', 'sum_max', 'sum_min', 'path'])
58631


In [ ]:
# test data 저장
with open("dic_minmax_test.pickle", "wb") as handle:
    pickle.dump(dic_minmax, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# 저장 잘 됐는지 확인
with open("dic_minmax_test.pickle", "rb") as handle:
    f = pickle.load(handle)

print( f.keys() )
print( len(f['path']) )

## image로 변환 및 svg파일로 저장
각 데이터가 시간에 따라 어떻게 변하는지 알아보고 저장하자.

메모리의 한계로 인해 train data 58631개 데이터 중 20000개의 데이터를 적용한다.

In [7]:
# 시간에 따라 x, y, z, 각 행의 합의 변화를 그리고 그 이미지를 저장하는 메소드
def make_plot(df) :
    fig = plt.figure(figsize=(32,1), dpi=64, tight_layout=True, frameon=False)
     # 공백 없애기
    plt.margins(0, 0, tight=True)
    plt.axis('off')
    plt.xticks([])
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)
    
    plt.scatter(x=df.wavelength, y=df['x'], s=4, alpha=0.7, color='midnightblue')
    plt.scatter(x=df.wavelength, y=df['y'], s=4, alpha=0.7, color='chartreuse')
    plt.scatter(x=df.wavelength, y=df['z'], s=4, alpha=0.7, color='mediumvioletred')
    plt.scatter(x=df.wavelength, y=df['sum'], s=1, alpha=0.7, color='crimson')
    fig.canvas.draw()
    plt.imshow(np.array(fig.canvas.renderer._renderer)[:,:])
    plt.savefig(path + filename[:-4] + '.svg',dpi=64,transparent=True,bbox_inches='tight')

In [2]:
train_path = 'E:/anomoly_detection_IOT_sensor/iotAnomaly_train'
i = 0
dic_minmax = {'x_max' : [], 'y_max' : [], 'z_max' : [],
              'x_min' : [], 'y_min' : [], 'z_min' : [],
              'sum_max' : [], 'sum_min' : [], 'path' : [] }
for (path, dir, files) in os.walk(train_path):
    print(path)
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.csv':
            df = pd.read_csv("%s/%s" % (path, filename))
            df['sum'] = df['x'] + df['y'] + df['z']
            
            make_plot(df)
            
            i+=1
            if i == 20000:
                break

E:/anomoly_detection_IOT_sensor/iotAnomaly_train
E:/anomoly_detection_IOT_sensor/iotAnomaly_train\csv_files


### test 파일에도 적용

8744개이기에 모두 저장

In [1]:
test_path = 'E:/anomoly_detection_IOT_sensor/iotAnomaly_test'
dic_minmax = {'x_max' : [], 'y_max' : [], 'z_max' : [],
              'x_min' : [], 'y_min' : [], 'z_min' : [],
              'sum_max' : [], 'sum_min' : [], 'path' : [] }
for (path, dir, files) in os.walk(test_path):
    print(path)
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.csv':
            try :
                df = pd.read_csv("%s/%s" % (path, filename))
                df['sum'] = df['x'] + df['y'] + df['z']
                make_plot(df)
            except KeyError as e :
                print("%s/%s" % (path, filename))
                break
